In [19]:
import pyspark
from pyspark.sql import SparkSession
from operator import add
from pyspark import SparkConf
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.6:7077") \
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","300s")\
        .config("spark.executor.cores",4)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .appName("Group2")\
        .getOrCreate()
sc = spark_session.sparkContext
spark = SparkSession(sc)

In [17]:
reddit_data = spark.read.json("hdfs://192.168.2.6:9000/user/ubuntu/input/RC_2010-01")
#print(reddit_data.count())
#reddit_data2 = reddit_data
#reddit_data.extend([reddit_data2 , reddit_data2])
#print(reddit_data.count())

In [3]:
reddit_data.first()


Row(archived=True, author='David_ungerer', author_flair_css_class=None, author_flair_text=None, body='Good rant, stop looking for a mass movement, if one appears fine, but first change yourself.\nWhile at 54 still a work in process, I am a vegetarian (see corporate tainted food ect.) I shop at farmers markets, small shops and co-ops (member owned) I live in a urban space for two of 900 sq ft. I ride a bike and take MTA. I bank at a Federal Credit Union (small, local, member owned. Most importantly, I choose to live simply.\nAs example . . . Go to a corporate drug store (the checkout girl has no health or retirement  and has to stock shelves and sweep floors for minimum wage or less if the corporation could, and is concerned abouting chatting about the community becouse of the boss.) or ride a mile to an owner run drug store (she welcome me by name, asks about my family and work or the community. She provides health and retirement for her workers.)\nEvery day and every doller I spend I 

In [4]:
from pyspark.sql import functions as F 
import time

In [ ]:
start = time.time()

reddit_data_2 = reddit_data.
end = 

In [5]:
reddit_data.take(5)

[Row(archived=True, author='David_ungerer', author_flair_css_class=None, author_flair_text=None, body='Good rant, stop looking for a mass movement, if one appears fine, but first change yourself.\nWhile at 54 still a work in process, I am a vegetarian (see corporate tainted food ect.) I shop at farmers markets, small shops and co-ops (member owned) I live in a urban space for two of 900 sq ft. I ride a bike and take MTA. I bank at a Federal Credit Union (small, local, member owned. Most importantly, I choose to live simply.\nAs example . . . Go to a corporate drug store (the checkout girl has no health or retirement  and has to stock shelves and sweep floors for minimum wage or less if the corporation could, and is concerned abouting chatting about the community becouse of the boss.) or ride a mile to an owner run drug store (she welcome me by name, asks about my family and work or the community. She provides health and retirement for her workers.)\nEvery day and every doller I spend I

In [20]:
import time 
start = time.time()
sub_count_mr = reddit_data.rdd \
    .map(lambda comment: (comment['subreddit'], 1)) \
    .reduceByKey(lambda accum, n: accum + n) \
    .collect()

end = time.time()


In [22]:
print(start)
print(end)
print(end-start)
print(len(sub_count_mr))
print(sorted(sub_count_mr, key=lambda sub_cnt: sub_cnt[1], reverse=True)[0:20])
#29.373241662979126
#28.87182331085205

1647892035.273016
1647892064.7076204
29.43460440635681
2313
[('AskReddit', 623825), ('reddit.com', 300476), ('pics', 198009), ('politics', 146922), ('IAmA', 135360), ('WTF', 114836), ('gaming', 113877), ('funny', 80320), ('atheism', 71549), ('science', 70028), ('programming', 66474), ('worldnews', 65451), ('technology', 49710), ('DoesAnybodyElse', 37237), ('Music', 26250), ('relationship_advice', 23163), ('fffffffuuuuuuuuuuuu', 20700), ('Economics', 19635), ('videos', 18558), ('trees', 18314)]


In [20]:
import string
import time
start = time.time()
def screamer_sub_mapper(comment):
    n_upper = len(list(filter(lambda c: c in string.ascii_uppercase, comment['body'])))
    n_alpha = len(list(filter(lambda c: c in string.ascii_letters, comment['body'])))
    return (comment['subreddit'], (n_upper, n_alpha))

def screamer_reducer(value_list):
    total_upper = 0
    total_alpha = 0
    for value in value_list:
        (n_upper, n_alpha) = value
        total_upper += n_upper
        total_alpha += n_alpha
    screamer_score = total_upper / total_alpha if total_alpha else 0
    return (screamer_score, len(value_list))
    
screamer_subs = reddit_data.rdd \
    .map(screamer_sub_mapper) \
    .groupByKey() \
    .mapValues(screamer_reducer) \

screamer_subs.cache()
screamer_subs.filter(lambda sub_rval: sub_rval[1][1] > 1000).sortBy(lambda sub_rval: sub_rval[1][0], False).take(3)
end = time.time()
print(start)
print(end)
print(end-start)

1647894707.283127
1647894777.0631738
69.78004670143127


In [22]:
#one core
one = 74.00914692878723
#two cores
two = 70.34418892860413
#three cores
three = 70.31279945373535
#four cores
four = 69.78004670143127

In [28]:
import matplotlib.pyplot as plt

fig = plt.figure()
x = [1,2,3,4]
y = [one,two,three,four]

plt.plot(x,y

plt.xlabel("seconds")
plt.ylabel("Y axis label")

AttributeError: 'Figure' object has no attribute 'set_xlabel'

<Figure size 432x288 with 0 Axes>